# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

In [8]:
df_aleatorio = pd.read_csv("instancias_aleatorias/indice.csv")

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.

- **VMC** : vecino mas cercano
- **AGM** : arbol generador minimo
- **I** : incersion
- **tabu falta** : tabu

In [42]:
def correr_experimento(metodo, archivo_instancia,parametros_tabu):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    cmds = ["../tp2", metodo] + parametros_tabu
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(cmds , stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(f"Hubo un error en la experimentacion para el algoritmo: {metodo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    costo = process.stdout.read() 
    #print(process.stdout.read()) # resultado del metodo
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    costo = costo.split(" ")[1]
    costo = costo.split("\n")[0]
    res = (tiempo_de_ejecucion,float(costo))
    
    return res;




In [32]:
correr_experimento("AGM-TS1",  "instancias_salida/berlin52.txt", ["50","50","50","50"])


(14.2955, '10402')

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [74]:
experimentos = [];
experimentos

[]

# Berlin

## berlin con Tabu 

In [60]:
data = "berlin"
n = 52
archivo_berlin = "instancias_salida/berlin52.txt"

In [75]:
Metodos = ["AGM-TS1","AGM-TS2","VMC-TS1","VMC-TS2","I-TS1","I-TS2"]
max_iter = [20,50,100,150]
rang_iter = [10,50,100,150] # tiene sentido si es menor que max
porcentaje_vecinos = [10,30,50,100]
largo_lista = [10,50,100]


for metodo in Metodos:    
    for m in max_iter:
        for r in rang_iter :
            for p in porcentaje_vecinos:
                for T in largo_lista :
                    experimentos.append([data, n ,metodo, archivo_berlin,[str(T),str(m),str(r),str(p)]]);


## Berlin sin tabu

In [76]:
experimentos.append([data, n ,"AGM", archivo_berlin,[str(0),str(0),str(0),str(0)]]);
experimentos.append([data, n ,"VMC", archivo_berlin,[str(0),str(0),str(0),str(0)]]);
experimentos.append([data, n ,"I", archivo_berlin,[str(0),str(0),str(0),str(0)]]);

## Experimento aleatorio


In [40]:
for i in range(0, df_aleatorio.shape[0]):
    fila = df_aleatorio.iloc[i];
    experimentos.append([fila["dataset"], fila["n"],"AGM", fila["archivo"],0,0,0,0]);
    experimentos.append([fila["dataset"], fila["n"],"VMC", fila["archivo"],0,0,0,0]);
    experimentos.append([fila["dataset"], fila["n"],"I", fila["archivo"],0,0,0,0]);    

In [6]:
for i in range(0, df_aleatorio.shape[0]):
    fila = df_aleatorio.iloc[i];
    experimentos.append([fila["dataset"], fila["n"],"I", fila["archivo"]]); 

In [ ]:
## Tabu search

In [50]:
for i in range(0, df_aleatorio.shape[0]):
    fila = df_aleatorio.iloc[i];
    experimentos.append([fila["dataset"], fila["n"],"AGM-TS1", fila["archivo"]]);
    experimentos.append([fila["dataset"], fila["n"],"AGM-TS2", fila["archivo"]]);
    
    experimentos.append([fila["dataset"], fila["n"],"VMC-TS1", fila["archivo"]]);
    experimentos.append([fila["dataset"], fila["n"],"VMC-TS2", fila["archivo"]]);

    
    #     experimentos.append([fila["dataset"], fila["n"],"I", fila["archivo"]]); 

## TABU

In [77]:
columnas = ["dataset", "n", "metodo", "tiempo","costo","max_iter","rang_iter","percentage","largolista"];


filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    costos = []
    resultados = []
    for i in range(0, T):
        exp = correr_experimento(experimento[2], experimento[3],experimento[4])
        t = exp[0]
        c = exp[1]
        costos.append(c)
        tiempos.append(t);
    costo = np.median(costos)
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], tiempo,costo,
                  experimento[4][0],experimento[4][1],experimento[4][2],experimento[4][3]]);

'Experimento: 1155/1155'

In [78]:
df_resultados = pd.DataFrame(filas, columns=columnas);

df_resultados.to_csv("resultados/resultado.csv", index=False, header=True);

In [79]:
df_resultados

,dataset,n,metodo,tiempo,costo,max_iter,rang_iter,percentage,largolista
0,berlin,52,AGM-TS1,2.193130,10402.0,10,20,10,10
1,berlin,52,AGM-TS1,2.021480,10402.0,50,20,10,10
2,berlin,52,AGM-TS1,2.033360,10402.0,100,20,10,10
3,berlin,52,AGM-TS1,2.579850,10402.0,10,20,10,30
4,berlin,52,AGM-TS1,2.550410,10402.0,50,20,10,30
...,...,...,...,...,...,...,...,...,...
1150,berlin,52,I-TS2,51.968000,15771.0,50,150,150,100
1151,berlin,52,I-TS2,52.164500,15771.0,100,150,150,100
1152,berlin,52,AGM,0.124531,10402.0,0,0,0,0
1153,berlin,52,VMC,0.078734,8980.0,0,0,0,0


In [9]:
df_viejo = pd.read_csv("resultados/resultado.csv")
df_viejo = df_viejo[df_viejo["metodo"] != "I"]
df_viejo

,dataset,n,metodo,tiempo,costo
0,dataset_aleatorias,10,AGM,0.018330,1231
1,dataset_aleatorias,10,VMC,0.008803,874
3,dataset_aleatorias,15,AGM,0.023357,1396
4,dataset_aleatorias,15,VMC,0.012061,1235
6,dataset_aleatorias,20,AGM,0.033290,1129
...,...,...,...,...,...
261,dataset_aleatorias,190,VMC-TS2,139.429000,2915
262,dataset_aleatorias,195,AGM-TS1,209.213000,8970
263,dataset_aleatorias,195,AGM-TS2,139.678000,8998
264,dataset_aleatorias,195,VMC-TS1,232.482000,3337


In [10]:
df_save = df_viejo.append(df_resultados,ignore_index = True)

In [11]:
df_save.to_csv("resultados/resultado.csv", index=False, header=True);

In [13]:
df_save

,dataset,n,metodo,tiempo,costo
0,dataset_aleatorias,10,AGM,0.018330,1231
1,dataset_aleatorias,10,VMC,0.008803,874
2,dataset_aleatorias,15,AGM,0.023357,1396
3,dataset_aleatorias,15,VMC,0.012061,1235
4,dataset_aleatorias,20,AGM,0.033290,1129
...,...,...,...,...,...
261,dataset_aleatorias,175,I,1465.460000,9003
262,dataset_aleatorias,180,I,1611.590000,9952
263,dataset_aleatorias,185,I,1823.510000,6076
264,dataset_aleatorias,190,I,1904.760000,7891
